In [17]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import time
from random import randint

In [18]:
options = webdriver.ChromeOptions()
options.add_argument("download.default_directory=" + os.getcwd() + "/Downloads")


Process Flow
-------------------
Search professors from start_page by name
Click Search
Find the first reseach item in the research section
Determine type of publication
Get Abstract if Abstract exists, if not skip to next
Click citation download link
Select option (1 = RIS, 2 = BibTex, 3 = Plain text)
Select option  (1 = citation 2 = citation and abstract)
Download
Go to most recent downloaded file
Read text from that file
Place citation into variable
If abstract exists for citation place into abstract variable if no abstract found insert "No abstract found."

<h2>To do</h2>

faculty,citation,title,type,abstract,book,journal,date,collabs,publisher,edition,series,volume,issue,chapter,pages,issn,isbn,handle,url



In [19]:
def google_scholar_scraper(record,driver):
    
    # Set driver to go to start_page and search a given name with valid scholar profile    
    google_scholar = "https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=&btnG="
    driver.get(google_scholar)

    time.sleep(randint(4,7))
    
    # Whether a profile is found is set to false by default
    profile_found = False
    search_bar = driver.find_element_by_class_name('gs_in_txt')
    search_bar.clear()
    search_bar.send_keys("\"" + record.faculty.strip() + "\"" + " CSUN")
    search_bar.send_keys(Keys.ENTER)
    
    # If there's search results
    if not driver.find_elements_by_xpath('//*[@id="gsc_sa_ccl"]/div/p[1]'):
        # A profile was found
        profile_found = True
    # If there's no search results let's try again
    else:
        # A little wait
        time.sleep(randint(4,7))
        search_bar = driver.find_element_by_class_name('gs_in_txt')
        search_bar.clear()
        search_bar.send_keys("\"" + record.faculty.strip() + "\"" + " Northridge")
        search_bar.send_keys(Keys.ENTER)
        if not driver.find_elements_by_xpath('//*[@id="gsc_sa_ccl"]/div/p[1]'):
            profile_found = True
            
    # If the profile was found
    if profile_found:
        time.sleep(randint(4,7))
        view_profile_link = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[1]/div/div/h3/a').get_attribute("href")
        citation_collector(record, view_profile_link)
    else:
        f = open("google_scholar_no_results.csv", "a")
        f.write(record.faculty.strip() + ",")
        f.write(record.college + ",")
        f.write(record.department + ",")
        f.write(record.url + "\n")
        f.close()
        
        

In [20]:
def citation_collector(record, url):
    
    driver = webdriver.Chrome()
    driver.implicitly_wait(15)
    driver.get(url)
    
    # To get all the links we need to click the Show More multiple times to show the whole links 
    gs_bpf_wrapper = driver.find_element_by_id("gsc_bpf_more")

    while not gs_bpf_wrapper.get_attribute("disabled"):
        gs_bpf_wrapper.click()
        time.sleep(3)

    #pull the first citation for the page
    view_citation_links = driver.find_elements_by_class_name('gsc_a_at')

    citation_file = open('google_scholar_citations.csv', "a")

    for link in view_citation_links:

        not_his_paper = False

        citation = ""
        title = link.text
        paper_type = ""
        book = ""
        journal = ""
        date = ""
        authors = ""
        publisher = ""
        edition = ""
        series = ""
        volume = ""
        issue = ""
        chapter = ""
        pages = ""
        issn = ""
        isbn = ""
        handle = ""


        time.sleep(randint(4,6))
        
        link.click()
        try:
            url = driver.find_element_by_xpath('//*[@id="gsc_vcd_title"]/a').get_attribute("href")
        except NoSuchElementException:
            url = ""
        
        
        time.sleep(randint(4,6))

        citation_table = driver.find_element_by_id("gsc_vcd_table")
        citation_table_data_objects= citation_table.find_elements_by_class_name("gs_scl")

        data_category = citation_table_data_objects[0].find_element_by_tag_name("div")

        if data_category.text == "Authors":

            i = 0
            for data in citation_table_data_objects:

                data_category = citation_table_data_objects[i].find_element_by_tag_name("div")

                citation_title = driver.find_element_by_id('gsc_vcd_title')
                title = citation_title.text

                if data_category.text == "Authors":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    if "|" in data_field.text:
                        authors = data_field.text
                        authors.replace("|", " ")
                    else:
                        authors = data_field.text
                    # what second condition was i going to check here?
                    if not record.faculty.strip() in data_field.text:
                        not_his_paper = True
                elif data_category.text == "Publication date":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    date = data_field.text
                elif data_category.text == "Publisher":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    publisher = data_field.text
                elif data_category.text == "Description":
                    # this line of code grabs the description
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    data = data_field.text

                    # here is where we want to go through the description and
                    # replace every instance of "\n" with " "
                    desc = data.split("\n")
                    description = ""
                    for piece in desc:
                        description += piece + " "
                elif data_category.text == "Journal":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    if "|" in data_field.text:
                        journal = data_field.text
                        journal.replace("|", " ")
                    else:
                        journal = data_field.text
                    journal = data_field.text
                elif data_category.text == "Issue":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    issue = data_field.text
                elif data_category.text == "Volume":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    volume = data_field.text
                elif data_category.text == "Pages":
                    data_field = data.find_element_by_class_name("gsc_vcd_value")
                    pages = data_field.text
                i = i + 1

            citation_file.write(record.faculty + "|" + citation + "|" + title + "|" + paper_type + "|")
            citation_file.write(description + "|" + book + "|" + journal + "|" + date + "|")
            citation_file.write(authors + "|" + publisher + "|" + edition + "|" + series + "|")
            citation_file.write(volume + "|" + issue + "|" + chapter + "|" + pages + "|")
            citation_file.write(issn + "|" + isbn + "|" + handle + "|" + url + "\n")

        elif data_category.text == "Inventors":
            pass

        elif not_his_paper:
            print("Error: Publication ", i , " did not get captured by program; new decision tree required.")

        close_table = driver.find_element_by_id("gs_md_cita-d-x")     
        close_table.click()
        time.sleep(randint(4,6))

    citation_file.close()
    driver.close()
    

[x] view_content

[] field_items

[] catalog_content

[] accordion

In [21]:
view_content = pd.read_csv("field_items.csv")
view_content.head()
print(view_content.columns)

Index(['faculty', 'college', 'department', 'url'], dtype='object')


In [22]:
view_content.iloc[258]

faculty                                        Markar Melkonian
college                                   College of Humanities
department                             Department of Philosophy
url            https://www.csun.edu/humanities/philosophy/le...
Name: 258, dtype: object

last person  Nadia (Nadejda) Fedortsova

In [16]:
try:
    driver = webdriver.Chrome()
    driver.implicitly_wait(15)
    for i in range(258,view_content.shape[0]):
        google_scholar_scraper(view_content.iloc[i],driver)
finally:
    driver.close()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="gsc_vcd_title"]/a"}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=73.0.3683.68 (47787ec04b6e38e22703e856e101e840b65afe72),platform=Linux 4.15.0-50-generic x86_64)
